In [1]:
# DEPENDENCIES
import glob
#from shutil import copyfile
import numpy as np
import pandas as pd
#import logomaker
# LOCAL IMPORTS
#from indexing_classes import GPCRDBIndexing
import sse_func
import matplotlib.pyplot as plt
import template_finder as tf
from indexing_classes import StAlIndexing
def find_pdb(name, pdb_folder):
    identifier = name.split("-")[0]
    target_pdb = glob.glob(f"{pdb_folder}/*{identifier}*.pdb")[0]
    return target_pdb

In [2]:
valid_collection = pd.read_pickle("../valid_collection.pkl")

In [3]:
for i, gain in enumerate(valid_collection.collection[:10]):
    #if "A0A6J3E5X4" not in gain.name:
    #    continue
    #print(i)
    file_prefix = f"../test_stal_indexing/t_all_{i}"
    print("_"*30, f"\n{i} {gain.name}")
    #for i, hel in enumerate(gain.sda_helices):
    #    print(f"H#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    #for i, hel in enumerate(gain.sdb_sheets):
    #    print(f"S#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    element_intervals, element_centers, residue_labels, unindexed_elements, params = tf.assign_indexing(gain, 
                                                                                                file_prefix=file_prefix, 
                                                                                                gain_pdb=find_pdb(gain.name, '../all_pdbs'), 
                                                                                                template_dir='../r2_template_pdbs/', 
                                                                                                debug=False, 
                                                                                                create_pdb=False,
                                                                                                hard_cut={"S2":7,"S6":3,"H5":3},
                                                                                                patch_gps=True
                                                                                                )
    split_modes = {
        0:"No Split.",
        1:"Split by coiled residue.",
        2:"Split by disordered residue.",
        3:"Split by Proline/Glycine",
        4:"Split by hard cut.",
        5:"Overwrite by anchor priority."
    }
    if params["split_mode"] > 0:
        print(params["split_mode"], split_modes[params["split_mode"]])
    #print(element_intervals, element_centers, residue_labels, unindexed_elements, sep="\n")


______________________________ 
0 A0A6J3E5X4-A0A6J3E5X4_AYTFU-LOWQUALITYPROTEIN:cadherinEGFLAGseven-passG-typereceptor2-Aythya_fuligula
1 Split by coiled residue.
______________________________ 
1 A0A6J3FJ26-A0A6J3FJ26_SAPAP-cadherinEGFLAGseven-passG-typereceptor2isoformX1-Sapajus_apella
1 Split by coiled residue.
______________________________ 
2 A0A6J3FJG3-A0A6J3FJG3_SAPAP-cadherinEGFLAGseven-passG-typereceptor2isoformX3-Sapajus_apella
1 Split by coiled residue.
______________________________ 
3 A0A6J3FK19-A0A6J3FK19_SAPAP-cadherinEGFLAGseven-passG-typereceptor2isoformX2-Sapajus_apella
1 Split by coiled residue.
______________________________ 
4 A0A6J3GH67-A0A6J3GH67_SAPAP-cadherinEGFLAGseven-passG-typereceptor1isoformX1-Sapajus_apella
1 Split by coiled residue.
______________________________ 
5 A0A6J3GHF3-A0A6J3GHF3_SAPAP-cadherinEGFLAGseven-passG-typereceptor1isoformX2-Sapajus_apella
1 Split by coiled residue.
______________________________ 
6 A0A6J3GHW0-A0A6J3GHW0_SAPAP-cadherinEG

In [4]:
identifier = gain.name.split('-')[0]

out = tf.run_command(f"find ../sigmas/sigma_2/ -name \"*{identifier}*stride\"")
print("subl", out)
print("pymol", find_pdb(identifier, '../all_pdbs'))
print(gain.sequence[717-gain.start:725-gain.start])

subl ../sigmas/sigma_2/A0A6J3IT34-A0A6J3IT34_SAPAP-cadherinEGFLAGseven-passG-typereceptor3isoformX2-Sapajus_apella.sigma_2.stride

pymol ../all_pdbs/A0A6J3IT34.pdb
['V' 'A' 'V' 'F' 'H' 'G' 'R' 'N']


In [5]:
print(element_intervals, element_centers, residue_labels, unindexed_elements, params, sep="\n")

{'H1': [452, 466], 'H2': [473, 489], 'H3': [495, 514], 'H4': [527, 540], 'H5': [542, 554], 'H6': [562, 579], 'S1': [588, 592], 'S2': [595, 602], 'S3': [611, 614], 'S4': [633, 637], 'S5': [677, 684], 'S6': [708, 710], 'S7': [714, 721], 'S8': [729, 739], 'S9': [750, 755], 'S10': [765, 767], 'S11': [771, 776], 'S12': [780, 787], 'S13': [789, 796], 'GPS': [787, 789]}
{'H1.50': 462, 'H2.50': 483, 'H3.50': 497, 'H4.50': 540, 'H5.50': 548, 'H6.50': 570, 'S1.50': 588, 'S2.50': 597, 'S3.50': 614, 'S4.50': 634, 'S5.50': 678, 'S6.50': 708, 'S7.50': 715, 'S8.50': 735, 'S9.50': 751, 'S10.50': 766, 'S11.50': 771, 'S12.50': 783, 'S13.50': 794, 'GPS': 788}
{'H1.40': 452, 'H1.41': 453, 'H1.42': 454, 'H1.43': 455, 'H1.44': 456, 'H1.45': 457, 'H1.46': 458, 'H1.47': 459, 'H1.48': 460, 'H1.49': 461, 'H1.50': 462, 'H1.51': 463, 'H1.52': 464, 'H1.53': 465, 'H1.54': 466, 'H2.40': 473, 'H2.41': 474, 'H2.42': 475, 'H2.43': 476, 'H2.44': 477, 'H2.45': 478, 'H2.46': 479, 'H2.47': 480, 'H2.48': 481, 'H2.49': 482, 

In [6]:
#offset information
def find_offsets(fasta_file, accessions, sequences):
    # searches through the accessions in the big sequence file,
    # finds the start for the provided sequence
    with open(fasta_file,"r") as fa:
        fa_data = fa.read()
        fasta_entries = fa_data.split(">")
    seqs = []
    headers = []
    offsets = []
    for seq in fasta_entries:
        # Fallback for too short sequences
        if len(seq) < 10: 
            continue
        data = seq.strip().split("\n")
        headers.append(data[0].split("|")[1]) # This is only the UniProtKB Accession Number and will be matched EXACTLY
        seqs.append("".join(data[1:]))

all_accessions = [gain.name.split("-")[0] for gain in valid_collection.collection]
all_sequences = ["".join(gain.sequence) for gain in valid_collection.collection]

fasta_offsets = find_offsets("/home/hildilab/projects/GPS_massif/uniprot_query/agpcr_celsr.fasta", 
                                 all_accessions, 
                                 all_sequences)

In [7]:
stal_indexing = StAlIndexing(valid_collection.collection, 
                             prefix="../test_stal_indexing/t_class", 
                             pdb_dir='../all_pdbs/',  
                             template_dir='../r2_template_pdbs/', 
                             fasta_offsets=fasta_offsets,
                             n_threads=6,
                             debug=False)

"""stal_indexing = StAlIndexing(valid_collection.collection[200:300], 
                             prefix="../test_stal_indexing/t_class", 
                             pdb_dir='../all_pdbs/',  
                             template_dir='../r2_template_pdbs/',
                             n_threads=6,
                             debug=False)"""


StAlIndexing: Assigning indexing via multiprocessing with 5 threads.
[DEBUG] split_segments : Last resort with 
	segment_sequence = array(['K', 'Y', 'L', 'G', 'T', 'T', 'V', 'G', 'T', 'A', 'V', 'I', 'S',
       'S', 'T', 'L', 'L'], dtype='<U1')
	sse = [685, 702]
[WARNING] No breakers, but Glycine detected in the target segment. Using this for splitting:
	segment_sequence = array(['K', 'Y', 'L', 'G', 'T', 'T', 'V', 'G', 'T', 'A', 'V', 'I', 'S',
       'S', 'T', 'L', 'L'], dtype='<U1')
	685-702 | G@688
[DEBUG] split_segments : Last resort with 
	segment_sequence = array(['D', 'S', 'D', 'L', 'L', 'E', 'S', 'S', 'S', 'E', 'A', 'L', 'K',
       'T', 'I', 'D', 'E', 'L', 'A', 'F'], dtype='<U1')
	sse = [591, 611]
[WARNING] HARD CUT BREAKER @ 597 between H5 | H6
[DEBUG] split_segments : Last resort with 
	segment_sequence = array(['D', 'S', 'D', 'L', 'L', 'E', 'S', 'S', 'S', 'E', 'A', 'L', 'K',
       'T', 'I', 'D', 'E', 'L', 'A', 'F'], dtype='<U1')
	sse = [591, 611]
[WARNING] HARD CUT BREAKER 

'stal_indexing = StAlIndexing(valid_collection.collection[200:300], \n                             prefix="../test_stal_indexing/t_class", \n                             pdb_dir=\'../all_pdbs/\',  \n                             template_dir=\'../r2_template_pdbs/\',\n                             n_threads=6,\n                             debug=False)'

In [8]:
import pickle
print(stal_indexing.indexing_dirs[0], stal_indexing.center_dirs[0], stal_indexing.intervals[0], sep="\n")
header, matrix = stal_indexing.construct_data_matrix(unique_sse=False)
stal_indexing.data2csv(header, matrix, "../test.stal_indexing.csv")

with open("stal_indexing.pkl","wb") as save:
    pickle.dump(stal_indexing, save)

{'H1.43': 501, 'H1.44': 502, 'H1.45': 503, 'H1.46': 504, 'H1.47': 505, 'H1.48': 506, 'H1.49': 507, 'H1.50': 508, 'H1.51': 509, 'H1.52': 510, 'H1.53': 511, 'H1.54': 512, 'H2.43': 516, 'H2.44': 517, 'H2.45': 518, 'H2.46': 519, 'H2.47': 520, 'H2.48': 521, 'H2.49': 522, 'H2.50': 523, 'H2.51': 524, 'H2.52': 525, 'H2.53': 526, 'H2.54': 527, 'H3.48': 533, 'H3.49': 534, 'H3.50': 535, 'H3.51': 536, 'H3.52': 537, 'H3.53': 538, 'H3.54': 539, 'H3.55': 540, 'H3.56': 541, 'H3.57': 542, 'H3.58': 543, 'H3.59': 544, 'H3.60': 545, 'H3.61': 546, 'H3.62': 547, 'H3.63': 548, 'H3.64': 549, 'H3.65': 550, 'H3.66': 551, 'H3.67': 552, 'H3.68': 553, 'H3.69': 554, 'H3.70': 555, 'H4.28': 559, 'H4.29': 560, 'H4.30': 561, 'H4.31': 562, 'H4.32': 563, 'H4.33': 564, 'H4.34': 565, 'H4.35': 566, 'H4.36': 567, 'H4.37': 568, 'H4.38': 569, 'H4.39': 570, 'H4.40': 571, 'H4.41': 572, 'H4.42': 573, 'H4.43': 574, 'H4.44': 575, 'H4.45': 576, 'H4.46': 577, 'H4.47': 578, 'H4.48': 579, 'H4.49': 580, 'H4.50': 581, 'H5.44': 583, 'H5.4